<a href="https://colab.research.google.com/github/Xelaro2304/MSB1015-Scientific-Programming/blob/main/Chess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

## Package installation

In [ ]:
!pip install -U ydata-profiling
!pip install berserk
!pip install optuna
!apt install stockfish -y
!pip install python-chess

## Import packages

In [ ]:
import gdown
import os
import berserk
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport
import re
import chess
import chess.engine

## Function definitions

In [ ]:
def plot_distribution(data, plot="hist", title=None, label=None, bins=30, show_stats=True, normalize=False, log = False, alpha = 0.7):
    """
    Plot a histogram (numeric) or count plot (categorical) for a single variable.

    Parameters:
    - data: array-like, the variable to plot
    - plot: "hist" for histogram, "count" for categorical count plot
    - title: optional plot title
    - label: optional x-axis label
    - bins: number of bins for histogram
    - show_stats: show mean/median/mode (only for histogram)
    - normalize: bool, whether to normalize frequencies/counts (0-1 or percentages)
    """
    plt.figure(figsize=(6,6))

    if plot == "hist":
        stat_type = 'density' if normalize else 'count'
        sns.histplot(data,
                     bins=bins,
                     kde=False,
                     color=sns.color_palette("colorblind")[0],
                     stat=stat_type,
                     alpha = alpha)

        if show_stats:
            mean_val = np.mean(data)
            median_val = np.median(data)
            mode_val = stats.mode(data, keepdims=True)[0][0]
            plt.axvline(mean_val, color=sns.color_palette("colorblind")[1], linestyle="--", linewidth=2.5, label=f"Mean = {mean_val:.2f}")
            plt.axvline(median_val, color=sns.color_palette("colorblind")[6], linestyle="--", linewidth=2.5, label=f"Median = {median_val:.2f}")
            plt.axvline(mode_val, color=sns.color_palette("colorblind")[3], linestyle="--", linewidth=2.5, label=f"Mode = {mode_val:.2f}")
            plt.legend()


        if log:
            plt.yscale('log')

        plt.ylabel("Density" if normalize else "Frequency")
        plt.xlabel(label if label else "Value")

    elif plot == "count":
        counts = data.value_counts(normalize=normalize)
        counts.plot(kind='bar', color=sns.color_palette("colorblind", len(counts)))
        plt.ylabel("Proportion" if normalize else "Count")
        plt.xlabel(label if label else "Category")

    else:
        raise ValueError("plot must be either 'hist' or 'count'")

    if title:
        plt.title(title)
    plt.tight_layout()
    plt.show()

def plot_winner_by(df, col, title=None, ylabel=None):
    """
    Plot horizontal percentage-stacked bar chart for chess game results.

    Parameters:
    - df: DataFrame containing 'winner' and the numeric column.
    - col: Column name to group by (e.g., 'start_time', 'increment', 'avg_rating').
    - title: Plot title.
    - ylabel: Label for y-axis.
    """
    df_plot = df.copy()

    if col == 'avg_rating':
        bins = [-np.inf, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000,
                2100, 2200, 2300, 2400, np.inf]
        labels = ["< 1000","1000-1100","1100-1200","1200-1300","1300-1400","1400-1500",
                  "1500-1600","1600-1700","1700-1800","1800-1900","1900-2000","2000-2100",
                  "2100-2200","2200-2300","2300-2400","> 2400"]
        df_plot[col] = pd.cut(df_plot[col], bins=bins, labels=labels, include_lowest=True)

    if col == 'rating_diff':
        df_plot['rating_diff_c2'] = np.where(df_plot['rating_diff'] > 0,
                                            "White higher rating", "White not-higher rating")
        # Count per category and winner
        count_df = df_plot.groupby(['rating_diff_c2', 'winner']).size().reset_index(name='count')
        count_pivot = count_df.pivot(index='rating_diff_c2', columns='winner', values='count').fillna(0)

        # Plot heatmap
        plt.figure(figsize=(8,6))
        sns.heatmap(count_pivot, annot=True, fmt='g', cmap='Greys', linewidths=0.8, linecolor='black', cbar=False,
                    annot_kws={"size": 15})
        plt.title(title if title else "Result of games by categorical difference in rating")
        plt.xlabel("Colour of winner")
        plt.ylabel("Rating before game")
        plt.show()
        return

    # --- Count per category and winner ---
    count_df = df_plot.groupby([col, 'winner']).size().reset_index(name='count')

    # --- Pivot for stacked percentage plot ---
    count_pivot = count_df.pivot(index=col, columns='winner', values='count').fillna(0)
    count_pct = count_pivot.div(count_pivot.sum(axis=1), axis=0)
    count_pct = count_pct[['white', 'draw', 'black']]  # desired stacking order

    # --- Define colors ---
    color_map = {'white': '#d9d9d9', 'draw': 'grey', 'black': 'black'}

    # --- Sort descending by the column ---
    count_pct = count_pct.sort_index(ascending=False)
    count_pivot = count_pivot.loc[count_pct.index]

    # --- Plot ---
    fig, ax = plt.subplots(figsize=(10, 8))
    count_pct.plot(kind='barh', stacked=True,
                   color=[color_map.get(c, 'grey') for c in count_pct.columns],
                   alpha=0.95, width=1, edgecolor='black', ax=ax)

    ax.set_xlabel("Share of wins")
    ax.set_ylabel(ylabel if ylabel else col)
    ax.set_title(title if title else f"Result of games by {col}")
    ax.legend(title="Colour of winner")
    ax.xaxis.set_major_formatter(plt.matplotlib.ticker.PercentFormatter(1.0))

    # --- Add counts in the middle of each segment ---
    for i, val in enumerate(count_pct.index):
        left = 0
        for winner in count_pct.columns:
            frac = count_pct.loc[val, winner]  # fraction for plotting
            value = count_pivot.loc[val, winner]  # raw count
            if value > 0:
                text_color = 'black' if winner == 'white' else 'white'
                ax.text(left + frac/2, i, int(value), ha='center', va='center',
                        color=text_color, fontsize=12)
                left += frac

    plt.tight_layout()
    plt.show()


## Data loading

In [ ]:
url = 'https://docs.google.com/uc?export=download&id=1lBXYMdZtKdMm4AtGWjJFjmBygUtn8w5y&confirm=t'
path = os.getcwd()
output = path + '/games.csv'
!wget -O "{output}" "{url}"

In [ ]:
games_df = pd.read_csv(output, sep=';')


# Exploratory data analysis

## Initial analysis

In [ ]:
games_df.head()

In [ ]:
games_df.shape

In [ ]:
games_df.info()

In [ ]:
games_df.isnull().sum()

In [ ]:
games_df.describe()

## Data anomaly

Everything seems normal except for that minimum white rating, which will be inspected further

In [ ]:
negative_rating = games_df["white_rating"]
negative_rating = negative_rating[negative_rating < 0]
print('Number of negative values:', len(negative_rating))
negative_rating.head()

There is another game with a negative value for a rating

In [ ]:
negative_rating_indices = list(negative_rating.index)
negative_rating_info = games_df.iloc[list(negative_rating_indices),]
negative_rating_info.head()

Will check the original values of the game by fetching it with game ID

In [ ]:
with open('./token') as f:

    token = f.read()
    token = token.strip()


session = berserk.TokenSession(token)

client = berserk.Client(session)

In [ ]:
negative_rating_games = list(negative_rating_info["id"])
corrected_ratings = []
for g in negative_rating_games:
    game = client.games.export(g, as_pgn=True)
    print(game)
    game = game.split('\n')
    corrected_ratings.append(int(game[9][11:15]))
print(corrected_ratings)

In [ ]:
games_positive_rtg = games_df
games_positive_rtg.loc[negative_rating_indices, 'white_rating'] = corrected_ratings
games_positive_rtg.loc[negative_rating_indices]

## Data report

In [ ]:
profile = ProfileReport(games_df,title="Games report")

profile.to_file("games_report.html")


In [ ]:
#!env BROWSER=firefox
#!open games_report.html
from IPython.display import HTML

# show an HTML file inside the notebook
HTML(filename="games_report.html")

## Player ratings


In [ ]:
plt.hexbin(games_df['white_rating'], games_df['black_rating'], gridsize=20, cmap='viridis')
plt.colorbar(label="Number of games")
plt.xlabel("White rating")
plt.ylabel("Black rating")
plt.title("Player ratings heatmap")
plt.show()


## Average ratings

In [ ]:
# Average rating
avg_rating = games_df
games_df['avg_rating'] = (games_df['white_rating'] + games_df['black_rating']) / 2
plot_winner_by(games_df, 'avg_rating', title="Result of games by average rating", ylabel="Average rating")


## Exploratory analysis conclusions

Things to notice:

1.   There seem to be some duplicated instances
1.   There are 400 unique increment codes, which seems problematic to use for classification
2.   The number of draws in winner is higher than the number of draws in victory status, will need to check that
1.   Winner classes are somewhat balanced, except for the amount of draws











#Preprocessing

## Duplicated instances

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats

game_ids = games_positive_rtg['id']
#duplicates = [i for i in game_ids if game_ids.count(i) > 1]
#print(duplicates)
print('Number of unique records:', len(games_positive_rtg['id'].unique()))
duplicate_counts = games_positive_rtg['id'].value_counts()
duplicate_ids = list(duplicate_counts[duplicate_counts > 1].index)
duplicate_counts = duplicate_counts[duplicate_counts > 1]
print('Total number of duplicated records:', sum(duplicate_counts))
print('Number of records duplicated:', len(duplicate_counts))
print('Duplicated ids:', duplicate_ids)

plot_distribution(duplicate_counts.values, 'hist', 'Number of Duplicates per Game ID', 'Amount of times duplicated', show_stats=False)


Out of the 20,058 records, 19113 are unique, but it is detecting only 813 replicates instead of 945

Repetition is mainly occuring in duplicates, although some of them are repeated 3-5 times

In [ ]:
games_unique = games_positive_rtg.drop_duplicates(keep='first')
print(f"Original rows: {len(games_positive_rtg)}, After removing duplicates: {len(games_unique)}")

Trying to remove duplicates values only removes ~400 of them, so I'll inspect further

In [ ]:
duplicate_sample = games_positive_rtg[games_positive_rtg['id'].isin(duplicate_ids[0:4])]
duplicate_sample.sort_values(by='id')

Some of the repeated instances have distinct values of "created_at" and "last_move_at", so I'll try removing it

In [ ]:
games_time_dropped = games_positive_rtg.drop(columns=['created_at', 'last_move_at'])
#games_time_dropped = games_positive_rtg.drop('last_move_at', axis=1)

games_unique = games_time_dropped.drop_duplicates(keep='first').reset_index(drop=True)
print(f"Original rows: {len(games_time_dropped)}, After removing duplicates: {len(games_unique)}")

All duplicates removed

## Convert increment codes

I'll try to handle the increment code in two ways:


1.   Separate time into minutes and time increment per move
2.   Classify each increment code into a time control



### Convert into initial time and increment move

In [ ]:
increment_code = games_unique['increment_code']
increment_code_split = [time.split('+') for time in increment_code]
print('Splitted increment codes:', increment_code_split)

#As minutes and increment
start_time = [int(minutes[0]) for minutes in increment_code_split]
print('Starting time in minutes:', start_time)

#bar_chart(list(games_unique.iloc()), start_time, 'Starting time per game ID')

increment = [int(seconds[1]) for seconds in increment_code_split]
print('Increment in seconds:', increment)



In [ ]:
plot_distribution(start_time, 'hist', 'Starting time distribution', 'Minutes', show_stats=True)

In [ ]:
plot_distribution(increment, 'hist', 'Increment distribution' 'Seconds', show_stats=True)

In [ ]:
start_time_df = pd.DataFrame(start_time, columns=['start_time'])
increment_df = pd.DataFrame(increment, columns=['increment'])
#check if there are games with 0 < start time < 1
#games_unique
under_minute = ((start_time_df < 1) & (start_time_df > 0)).sum()
print('Games with less than 1 minute of start time:', under_minute.iloc[0])


Most games are finish (no increment) and have 10 minutes as start time, with no game starting with less than a minute

In [ ]:
games_unique = pd.concat([games_unique, start_time_df], axis = 1)
games_unique = pd.concat([games_unique, increment_df], axis = 1)
games_unique.info()

In [ ]:
plot_winner_by(games_unique, 'start_time', title="Result of games by start time", ylabel="Start time (minutes)")

In [ ]:
plot_winner_by(games_unique, 'increment', title="Result of games by increment", ylabel="Increment (seconds)")


### Convert into increment codes

According to the data source, [Lichess](https://lichess.org/faq#time-controls), time controls are decided assuming a game length of 40 moves and assigning the following categories depending on the duration:

    ≤ 29s = UltraBullet
    ≤ 179s = Bullet
    ≤ 479s = Blitz
    ≤ 1499s = Rapid
    ≥ 1500s = Classical

In [ ]:
def set_time_control(minutes, increment):
    total_time = minutes*60+increment*40
    if total_time <= 29:
        return 'UltraBullet'
    elif total_time <= 179:
        return 'Bullet'
    elif total_time <= 479:
        return 'Blitz'
    elif total_time <= 1499:
        return 'Rapid'
    else:
        return 'Classical'

time_control = games_unique.apply(lambda x: set_time_control(x['start_time'], x['increment']), axis=1)
time_control_df = pd.DataFrame({'time_control': time_control})
time_control_df.info()


In [ ]:
plot_distribution(time_control, "count",'Time control count', 'Time control')

Too few blitz games to the point they are not even appreciated

In [ ]:
blitz = [i for i in time_control if i == 'Blitz']
print('Number of blitz games:', len(blitz))

Very few blitz games
Since there are too few it would cause troubles during the training and testing, so I dont consider this approach to be viable any longer and will not proceed with it

## Analysis of drawed games

Accordint to the stats report, not all games with a winner status of draw have a winner value of draw

In [ ]:
draws = games_unique[['victory_status','winner']]
draws = draws[games_unique['winner'] == 'draw']
not_draw = draws[draws['victory_status'] != 'draw']
plot_distribution(draws['victory_status'], 'count', 'Victory status of draws', 'Victory Status')

The alternative victory status  of drawed games is out of time, which makes sense since the game can result in a draw by insufficient winning material even when running out of time, so it is not a recording error

## Rating difference

I will also add the rating difference as a feature to see if it is useful for the predictions

In [ ]:
games_unique['rating_diff'] = games_unique['white_rating'] - games_unique['black_rating']

plt.figure(figsize=(6, 6))

# Violin plot with boxplot and points
sns.violinplot(y=games_unique['rating_diff'], inner=None, color="lightblue")  # violin
sns.boxplot(y=games_unique['rating_diff'], width=0.1, color="white", fliersize=0)

plt.title("Rating difference violing plot")
plt.ylabel("Rating difference")
plt.show()

In [ ]:
sns.histplot(games_unique['rating_diff'], kde=True, bins=50)
plt.xlabel("Rating difference (White - Black)")
plt.title("Distribution of rating differences")
plt.show()

In [ ]:
# Categorical rating difference
plot_winner_by(games_unique, 'rating_diff', title="Result of games by higher rating player in rating")

## Game Evaluation

Finally,as a simple approach to implement game information I am getting the final evaluation of the position from the algebraic notation of the game

This is a lengthy process, so an excel file with the processed results is provided while the cell is commented

### Removal of short games

To avoid having a short game in which one of the players made a sudden abandonment or ran out of time I will filter all the games that did not reach left the opening phase and not entered the middle game

While there is no clear division between the end of the opening and the start of the middle game, I will assume a that middlegame starts after move 15 (30 turns) and remove games shorter than that

In [ ]:
plot_distribution(games_unique['turns'], 'hist', 'Number of moves per game', 'Number of moves', bins=50)

In [ ]:
# Keep only rows with at least 30 moves
games_long = games_unique.copy()
games_long = games_long[games_long['moves'].str.split().str.len() >= 30]

games_long = games_long.reset_index(drop=True)
games_long.info()

### Game evaluation

In [ ]:
engine = chess.engine.SimpleEngine.popen_uci("/usr/games/stockfish")
def final_eval(moves, idx=None, depth=15):
    """
    Evaluate final position from White's perspective (positive = White better)
    as centipawn evaluation. Handles notation cleanup, invalid moves,
    and malformed strings.
    """
    if idx is not None and idx % 100 == 0:
        print(f"{idx} rows analyzed")

    # Clean move text: remove numbers, results, and dots
    moves = re.sub(r"\d+\.", "", moves)
    moves = re.sub(r"(1-0|0-1|1/2-1/2|\*)", "", moves)
    moves = moves.strip()

    board = chess.Board()

    # Try to play all moves
    for move in moves.split():
        try:
            board.push_san(move)
        except Exception:
            # If illegal move return determined value
            return 23.04

    # Run engine analysis
    info = engine.analyse(board, chess.engine.Limit(depth=depth))
    score = info["score"].pov(chess.WHITE).score(mate_score=20000)
    return score


games_processed = games_long.copy()
games_processed["final_eval"] = [
    final_eval(moves, idx=i) for i, moves in enumerate(games_processed["moves"])
]
engine.quit()
games_processed.head()

In [ ]:
'''
from google.colab import files

# Suppose your DataFrame is called df
games_processed.to_csv("games_processed.csv", index=False)  # Save to CSV file

# Download to your computer
files.download("games_processed.csv")
'''

In [ ]:
'''
url_processed = 'https://docs.google.com/uc?export=download&id=1mLUJ0b4z28_y8vngISKJxjhwIlbBwm9j'
output_processed = path + '/games_processed.csv'
!wget -O "{output}" "{url_processed}"

games_processed = pd.read_csv(output_processed, sep=',')
games_processed.head()
'''

In [ ]:
plot_distribution(games_processed['final_eval'], 'hist', 'Engine Evaluation distribution', 'Engine Evaluation', bins = 60, log =True)

In [ ]:
plt.figure(figsize=(10,6))
sns.violinplot(
    x='winner',
    y='final_eval',
    data=games_processed,
    inner='quartile',
    palette='colorblind'
)
plt.yscale('symlog', linthresh=100)
plt.xlabel('Game Result')
plt.ylabel('Engine Evaluation')
plt.title('Distribution of Engine Evaluations by Game Result')
plt.grid(True, alpha=0.3)
plt.show()

## Repeat exploratory analysis

In [ ]:
profile = ProfileReport(games_processed,title="Games after processing report")

profile.to_file("games_processed_report.html")
# show an HTML file inside the notebook
HTML(filename="games_processed_report.html")

# Modelling

## Data split

Drop unused features

In [ ]:
games_selected = games_processed.drop(['id',
                                        'turns',
                                        'increment_code',
                                        'victory_status',
                                        'white_id',
                                        'black_id',
                                        'moves',
                                        'opening_eco',
                                        'opening_name',
                                        'opening_ply',], axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split

def define_train_test(df):
    #Defines predictors and class variable and returns the train and test datasets
    #If submission = True, it returns X and y without splitting since it will be used for training new dataset

    target = 'winner'
    y = df[target]
    X = df.drop(target, axis = 1)

    #80/20 split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state = 42)

    print('Shape of Data (20%)')
    print("X_train shape : ", X_train.shape)
    print("y_train shape : ", y_train.shape)
    print("X_test shape : ", X_test.shape)
    print("y_test shape : ", y_test.shape)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = define_train_test(games_selected)

In [ ]:
games_selected.info()
X_train.info()

## Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

def encode (y_tra, y_te):
    encoder = LabelEncoder()
    y_tra = encoder.fit_transform(y_tra)
    y_te = encoder.transform(y_te)
    return y_tra, y_te, encoder

y_train, y_test, le = encode(y_train, y_test)

## Model optimization

In [ ]:
import optuna
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.model_selection import cross_val_score, StratifiedKFold
from optuna.samplers import TPESampler

def optimize_model(X_tr, y_tr):
    optuna.logging.set_verbosity(optuna.logging.WARNING)

    def logging_callback(study, frozen_trial):
        previous_best_value = study.user_attrs.get("previous_best_value", None)
        if previous_best_value != study.best_value:
            study.set_user_attr("previous_best_value", study.best_value)
            print(
                "Trial {} finished with best value: {} and parameters: {}. ".format(
                frozen_trial.number,
                frozen_trial.value,
                frozen_trial.params,
                )
            )

    def objective(trial, X, y):

        def rf_model(trial):
            #Objective function for bayesian hyperparameter optimization of a
            #Random Forest classifier using Optuna
            params = {
                "n_estimators": trial.suggest_int("n_estimators", 50, 250),
                "max_depth": trial.suggest_int("max_depth", 4, 10),
                "min_samples_split": trial.suggest_int("min_samples_split", 10, 30),
                "min_samples_leaf": trial.suggest_int("min_samples_leaf",  5, 20),
                "max_features": trial.suggest_int("max_features", 1,5),
                "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
                "random_state": 42

            }
            model = RandomForestClassifier(**params, n_jobs=-1)
            return model

        def lgbm_model(trial):
            #Objective function for bayesian hyperparameter optimization of a
            #Light Gradient-Boosting Machine using Optuna
            params = {
                "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1000),
                "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 1000),
                "random_state": 42,
                "subsample": trial.suggest_float("subsample", 0.5, 1.0),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
            }
            model = LGBMClassifier(**params, verbose=-1)
            return model

        def xgb_model(trial):
            #Objective function for bayesian hyperparameter optimization of an
            #eXtreme Gradient Boostin classifier using Optuna
            params = {
                "reg_alpha": trial.suggest_float("reg_alpha", 0, 1000),
                "reg_lambda": trial.suggest_float("reg_lambda", 0, 1000),
                "subsample": trial.suggest_float("subsample", 0.5, 1.0),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
                "eval_metric": "mlogloss",
                "random_state": 42
            }
            model = XGBClassifier(**params)
            return model

        # Select which model to use
        model_name = trial.suggest_categorical("model", ["RandomForest", "XGBoost", "Light Gradient-Boosting Machine"])

        if model_name == "RandomForest":
            model = rf_model(trial)
        elif model_name == "XGBoost":
            model = xgb_model(trial)
        else:
            model = lgbm_model(trial)

        # Evaluate with stratified cross-validation
        stratified_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        recall_scorer = make_scorer(recall_score, average='macro')
        score = cross_val_score(model, X, y, cv=stratified_cv, scoring=recall_scorer).mean()
        return score


    #Create optuna study
    sampler = TPESampler(seed=10)
    study = optuna.create_study(direction="maximize", sampler = sampler)
    #Optimize study
    study.optimize(lambda trial: objective(trial, X_tr, y_tr), n_trials=200, callbacks = [logging_callback])
    print(f"Best averaged recall: {study.best_value:.4f}")
    best_params = study.best_params.copy()
    print(f"Best hyperparameters: {best_params}")
    return best_params

optimized = optimize_model(X_train, y_train)


## Fitting and predictions

In [ ]:
def model_predict(params, X_tr, X_te, y_tr, y_te):
    best_params = params.copy()
    best_model_name = best_params.pop("model")

    if best_model_name == "RandomForest":
        best_model = RandomForestClassifier(**best_params, random_state=42, n_jobs=-1)
    elif best_model_name == "XGBoost":
        best_model = XGBClassifier(**best_params, eval_metric="logloss", random_state=42)
    else:  # Light Gradient-Boosting Machine
        best_model = LGBMClassifier(**best_params, max_iter=1000, random_state=42)

    best_model.fit(X_tr, y_tr)
    test_recall = best_model.score(X_te, y_te)
    print(f"Best model: {best_model_name}")
    print(f"Test set recall: {test_recall:.4f}")
    y_predictions = best_model.predict(X_te)
    y_probabilities = best_model.predict_proba(X_te)
    return best_model, y_predictions, y_probabilities

fitted_model, y_pred, y_pred_proba = model_predict(optimized, X_train, X_test, y_train, y_test)

#Results

## Feature importance

In [ ]:
def feature_importance(best_model, X):
    # Calculate feature importance

    importances = best_model.feature_importances_
    features = X.columns

    #Normalize importance
    importances = importances / importances.sum()

    # Put into a DataFrame for easy sorting
    feat_imp = pd.DataFrame({
        "Feature": features,
        "Importance": importances
    }).sort_values(by="Importance", ascending=False)

    # Plot
    plt.figure(figsize=(8, 5))
    plt.barh(feat_imp["Feature"], feat_imp["Importance"], color="skyblue")
    plt.gca().invert_yaxis()  # Most important at the top
    plt.xlabel("Importance")
    plt.title("Feature Importance")
    plt.show()


feature_importance(fitted_model, X_train)

## Classification metrics

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

def classification_metrics(best_model, X, y, encoder):
    """
    Plot confusion matrix and print classification metrics.
    """
    # Predict encoded labels
    y_pred = best_model.predict(X)

    # Decode both true and predicted labels
    y_true_decoded = encoder.inverse_transform(y)
    y_pred_decoded = encoder.inverse_transform(y_pred)

    labels = ['white', 'draw', 'black']

    # Confusion matrix (use decoded labels)
    cm = confusion_matrix(y_true_decoded, y_pred_decoded, labels=labels, normalize='true')
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)

    fig, ax = plt.subplots(figsize=(6, 6))
    disp.plot(cmap='Blues', ax=ax, colorbar=False)
    plt.title("Confusion Matrix")
    plt.show()

    # Classification report (also use decoded labels)
    print("\nClassification report:\n")
    print(classification_report(y_true_decoded, y_pred_decoded, target_names=labels))



classification_metrics(fitted_model, X_test, y_test, le)


## ROC curve

In [ ]:
from sklearn.metrics import roc_curve, auc
import numpy as np
from sklearn.preprocessing import LabelEncoder, label_binarize

def plot_multiclass_roc(y_true, y_proba, model, label_encoder):
    """
    Plots ROC curves for multi-class classification.
    y_true should be integer-encoded.
    y_pred_proba should be probability outputs with shape (n_samples, n_classes).
    """
    # Use the encoder’s class order to ensure consistency
    classes = label_encoder.classes_
    n_classes = len(classes)

    # Binarize the true labels according to the class order
    y_true_bin = label_binarize(y_true, classes=np.arange(n_classes))

    # Colorblind-friendly palette
    colors = sns.color_palette("colorblind", n_classes)

    plt.figure(figsize=(8, 6))

    # Per-class ROC curves
    for i, cls in enumerate(classes):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_proba[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2, color=colors[i], label=f'{cls} (AUC = {roc_auc:.2f})')

    # Macro-average ROC
    all_fpr = np.unique(
        np.concatenate([roc_curve(y_true_bin[:, i], y_proba[:, i])[0] for i in range(n_classes)])
    )
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_proba[:, i])
        mean_tpr += np.interp(all_fpr, fpr, tpr)
    mean_tpr /= n_classes
    roc_auc_macro = auc(all_fpr, mean_tpr)
    plt.plot(all_fpr, mean_tpr, color='navy', linestyle='-',
             label=f'Macro-average ROC (AUC = {roc_auc_macro:.2f})', lw=2)

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Multi-class ROC Curve')
    plt.legend(loc='lower right')
    plt.show()

plot_multiclass_roc(y_test, y_pred_proba, fitted_model, le)


## Precision Recall curve

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score

def plot_precision_recall_curve(y_true, y_proba, model, label_encoder):
    """
    Plots precision–recall curves for multi-class classification.

    Parameters:
    - y_true: encoded (numeric) labels
    - y_prob: predicted probabilities (n_samples × n_classes)
    - model: fitted classifier
    - label_encoder: fitted LabelEncoder
    """
    classes = label_encoder.classes_
    n_classes = len(classes)

    # Binarize true labels using the encoder’s order
    y_true_bin = label_binarize(y_true, classes=np.arange(n_classes))

    # Colorblind friendly palette
    colors = sns.color_palette("colorblind", n_classes)

    plt.figure(figsize=(8, 6))

    # Per-class PR curves
    for i, cls in enumerate(classes):
        precision, recall, _ = precision_recall_curve(y_true_bin[:, i], y_proba[:, i])
        ap = average_precision_score(y_true_bin[:, i], y_proba[:, i])
        plt.plot(
            recall, precision, lw=2, color=colors[i],
            label=f'{cls} (AP = {ap:.2f})'
        )

    # Macro-average curve
    all_recall = np.unique(np.concatenate([
        precision_recall_curve(y_true_bin[:, i], y_proba[:, i])[1] for i in range(n_classes)
    ]))
    mean_precision = np.zeros_like(all_recall)
    for i in range(n_classes):
        p, r, _ = precision_recall_curve(y_true_bin[:, i], y_proba[:, i])
        mean_precision += np.interp(all_recall, r[::-1], p[::-1])
    mean_precision /= n_classes
    mean_ap = np.mean([
        average_precision_score(y_true_bin[:, i], y_proba[:, i]) for i in range(n_classes)
    ])
    plt.plot(
        all_recall, mean_precision,
        color='black', linestyle='--', lw=2,
        label=f'Macro-average (AP = {mean_ap:.2f})'
    )

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Multi-class Precision–Recall Curve')
    plt.legend(loc='lower left')
    plt.grid(True, alpha=0.3)
    plt.show()

plot_precision_recall_curve(y_test, y_pred_proba, fitted_model, le)

# Second model

I want to retry it by removing redundant features from the input, so I will make a pipeline

In [ ]:
from google.colab import files

# Suppose your DataFrame is called df
games_processed.to_csv("games_processed.csv", index=False)  # Save to CSV file

# Download to your computer
files.download("games_processed.csv")

In [ ]:
games_selected_v2 = games_selected.drop(['avg_rating',
                                         'rated',
                                         'increment',], axis = 1)

In [ ]:
def run_model(df):
    X_train_v2, X_test_v2, y_train_v2, y_test_v2 = define_train_test(games_selected_v2)
    y_train_v2, y_test_v2, le_v2 = encode(y_train_v2, y_test_v2)
    optimized_v2 = optimize_model(X_train_v2, y_train_v2)
    fitted_model_v2, y_pred_v2, y_pred_proba_v2 = model_predict(optimized_v2, X_train_v2, X_test_v2, y_train_v2, y_test_v2)
    classification_metrics(fitted_model_v2, X_test_v2, y_test_v2, le_v2)
    plot_multiclass_roc(y_test_v2, y_pred_proba_v2, fitted_model_v2, le_v2)
    plot_precision_recall_curve(y_test_v2, y_pred_proba_v2, fitted_model_v2, le_v2)

    return fitted_model_v2, X_train_v2, X_test_v2, y_train_v2, y_test_v2, y_pred_v2, y_pred_proba_v2, le_v2

fitted_model_v2, X_train_v2, X_test_v2, y_train_v2, y_test_v2, y_pred_v2, y_pred_proba_v2, le_v2 = run_model(games_selected_v2)